In [8]:
import xml.etree.ElementTree

def updateContent(child: xml.etree.ElementTree, content:str):
    if (child.text is not None):
        content=content+child.text+" "
    if (child.tail is not None):
        content=content+child.tail+ " "
    return content
            
    
# Funnction to parse regulations only
def parseRegulation(root: xml.etree.ElementTree):
    '''
      Parses regulation based xml file
      :param root: xml node of type xml.etree.ElementTree
      :returns dictionary of keys and values
    '''
    xRefXternal=list()
    content=""
    modifiedYear="NA"
    regYear="NA"
    consolidationYear="NA"
    #dt=""
    #modFlag=False
    xmlDict=dict()
    
    
    for child in root.iter():
        #print (child.tag,child.text,child.tail)
        if child.tag=="XRefExternal":
            txt=""
            if child.text is None:
                if child[0].tail is not None:
                    txt=child[0].tail
                else:
                     txt=child[0].text
            else:
                txt=child.text
            xRefXternal.append(txt)
            content=updateContent(child,content)
            #content+=txt+" "
       # get modifed date 
        elif ("ModifiedDate" in child.tag):
         
            dateTag=child[0]
            #print(date[1].text)
            modifiedYear=dateTag[0].text
        elif (child.tag=="RegistrationDate"):
            dateTag=child[0]
            #print(date[1].text)
            regYear=dateTag[0].text
        elif (child.tag=="ConsolidationDate"):
            dateTag=child[0]
            #print(date[1].text)
            consolidationYear=dateTag[0].text 
        elif (child.tag=="InstrumentNumber"):
            xmlDict["instrumentNumber"]=child.text
        elif (child.tag=="ShortTitle"):
            xmlDict["shorttitle"]=child.text
        elif (child.tag=="RegulationMaker"):
            xmlDict["regulationmaker"]=child.text
        elif (child.tag=="LongTitle"):
            xmlDict["longtitle"]=child.text 
            content+=child.text+". "
        elif (child.tag=="TitleText"):
            if child.text is None:
                if len(child)>0  and child[0].text is not None:
                    content+=child[0].text+". "
            else:
                content+=child.text+". "
       
            
        elif child.tag=="MM" or child.tag=="DD" or child.tag=="YYYY"  or child.tag=="Label":
            continue;
        elif child.tag=="Repealed":
            print (child.text)
            return None # igonre reglations with repeal keywords
            
        else:
            content=updateContent(child,content)
            #print(child.tag,child.text)
            '''if (child.text is not None):
            
                content=content+child.text+" "
            if (child.tail is not None):
                content=content+child.tail+ " "'''
                
           
    xmlDict["modifiedyear"]=modifiedYear
    xmlDict["registrationyear"]=regYear
    xmlDict["consolidationyear"]=consolidationYear
    xmlDict["xrefxternal"]=xRefXternal
    xmlDict["content"]=content
    
    
    return (xmlDict)
        
##################
#    Acts Parser
#################        

def parseAct(root: xml.etree.ElementTree):
    '''
      Parses regulation based xml file
      :param root: xml node of type xml.etree.ElementTree
      : returns dictionary of keys and values
    '''
    xRefXternal=list()
    xRefInternal=list()
    content=""
    regYear="NA"
    consolidationYear="NA"
    regnalYears="NA"
     
    #dt=""
    #modFlag=False
    xmlDict=dict()
    
    
    for child in root.iter():
        #print (child.tag,child.get("stage"))
        if child.tag=="XRefExternal":
            txt=""
            if child.text is None:
                if child[0].tail is not None:
                    txt=child[0].tail
                else:
                     txt=child[0].text
            else:
                txt=child.text
            xRefXternal.append(txt)
            #content+=txt+" "
            content=updateContent(child,content)
       # get modifed date 
       
        elif ("Stages"==child.tag and child.get("stage")=="assented-to"):
            
            dateTag=child[0]
            #print(date[1].text)
            regYear=dateTag[0].text
        elif (child.tag=="Stages" and child.get("stage")=="consolidation"):
            dateTag=child[0]
            #print(date[1].text)
            consolidationYear=dateTag[0].text
        elif (child.tag=="RegnalYear"):
            regnalYears=child.text
        elif (child.tag=="ConsolidatedNumber"):
            xmlDict["ConsolidatedNumber"]=child.text
        elif (child.tag=="ShortTitle"):
            xmlDict["shorttitle"]=child.text
        elif (child.tag=="RunningHead"):
            xmlDict["runninghead"]=child.text
        elif (child.tag=="ConsolidatedNumber"):
            xmlDict["consolidatednumber"]=child.text
        elif (child.tag=="AnnualStatuteNumber"):
            xmlDict["annualstatutenumber"]=child.text
        elif (child.tag=="XRefInternal"):
            xRefInternal.append(child.text)
        elif (child.tag=="LongTitle"):
            if (child.text is not None): # funny sometimes there are two long titles 
                xmlDict["longtitle"]=child.text 
                content+=child.text+". "
        elif (child.tag=="TitleText"):
            if child.text is None:
                if len(child)>0  and child[0].text is not None:
                    content+=child[0].text+". "
            else:
                content+=child.text+". "
       
       
            
        elif child.tag=="MM" or child.tag=="DD" or child.tag=="YYYY"  or child.tag=="Label":
            continue;
        elif child.tag=="Repealed":
            print (child.text)
            return None # igonre reglations with repeal keywords
            
            
        else:
            #print(child.tag,child.text)
            content=updateContent(child,content)
            '''if (child.text is not None):
            
                content=content+child.text+" "
            if (child.tail is not None):
                content=content+child.tail+ " "'''
                
    
    
    xmlDict["registrationyear"]=regYear
    xmlDict["consolidationyear"]=consolidationYear
    xmlDict["regnalyears"]=regnalYears
    xmlDict["xrefxternal"]=xRefXternal
    xmlDict["xrefinternal"]=xRefInternal
    xmlDict["content"]=content
    
    
    return (xmlDict)
        
        
        
# Unit test: usage method


In [63]:
import os
import xml.etree.ElementTree as ET
lst = []
directory="C:/Users/amitp/OneDrive/Ryerson-DS/mrp/regulations"
#directory="..somepaths/acts"
count=0
n=0
for filename in os.listdir(directory):
    if filename.endswith(".xml") and (not filename.startswith("SI-")):
        #print (filename)
        path=os.path.join(directory, filename)
        tree=ET.parse(path)
        d=parseRegulation(tree.getroot())
        n=n+1
        d=parseAct(tree.getroot())
        if (d==None):
            #print (filename)
            count =count +1
        else:
            lst.append(d)
        #print(d)
#print(count)
#print (n)
       # d=parseAct(tree.getroot())
        #print(d)

[Repealed, SOR/2009-231, s. 7]
[Repealed, SOR/2009-231, s. 7]
[Revoked, SOR/93-401, s. 2]
[Revoked, SOR/93-401, s. 2]
[Repealed, SOR/2010-213, s. 1]
[Repealed, SOR/2010-213, s. 1]
[Repealed, SOR/2013-204, s. 1]
[Repealed, SOR/2013-204, s. 1]
[Revoked, SOR/81-201, s. 1]
[Revoked, SOR/81-201, s. 1]
[Repealed, SOR/82-671, s. 1]
[Repealed, SOR/82-671, s. 1]
[Repealed, SOR/89-521, s. 2]
[Repealed, SOR/89-521, s. 2]
[Repealed, SOR/2003-111, s. 2]
[Repealed, SOR/2003-111, s. 2]
[Repealed, SOR/80-577, s. 1]
[Repealed, SOR/80-577, s. 1]
[Repealed, SOR/80-738, s. 1]
[Repealed, SOR/80-738, s. 1]
[Repealed, SOR/2008-258, s. 1]
[Repealed, SOR/2008-258, s. 1]
[Revoked, SOR/93-401, s. 2]
[Revoked, SOR/93-401, s. 2]
[Repealed, SOR/2016-230, s. 261]
[Repealed, SOR/2016-230, s. 261]
[Repealed, SOR/2002-165, s. 1]
[Repealed, SOR/2002-165, s. 1]
[Revoked, SOR/90-306, s. 1]
[Revoked, SOR/90-306, s. 1]
[Revoked, SOR/86-999, s. 12]
[Revoked, SOR/86-999, s. 12]
[Revoked, SOR/81-176, s. 1]
[Revoked, SOR/81-176

[Revoked, SOR/85-126, s. 2]
[Revoked, SOR/85-126, s. 2]
[Revoked, SOR/78-842, s. 1]
[Revoked, SOR/78-842, s. 1]
[Repealed, SOR/2017-286, s. 34]
[Repealed, SOR/2017-286, s. 34]
[Revoked, SOR/79-406, s. 1]
[Revoked, SOR/79-406, s. 1]
[Repealed, SOR/2007-61, s. 1]
[Repealed, SOR/2007-61, s. 1]
[Repealed, SOR/2005-168, s. 2]
[Repealed, SOR/2005-168, s. 2]
[Repealed, SOR/2010-277, s. 78]
[Repealed, SOR/2010-277, s. 78]
[Repealed, SOR/2013-61, s. 1]
[Repealed, SOR/2013-61, s. 1]
[Repealed, SOR/2017-120, s. 2]
[Repealed, SOR/2017-120, s. 2]
[Repealed, SOR/2013-61, s. 6]
[Repealed, SOR/2013-61, s. 6]
[Repealed, SOR/2016-64, s. 1]
[Repealed, SOR/2016-64, s. 1]
[Repealed, SOR/2013-19, s. 6]
[Repealed, SOR/2013-19, s. 6]
[Revoked, SOR/88-130, s. 1]
[Revoked, SOR/88-130, s. 1]
[Repealed, SOR/2017-110, s. 2]
[Repealed, SOR/2017-110, s. 2]
[Repealed, SOR/2017-58, s. 42]
[Repealed, SOR/2017-58, s. 42]
[Repealed, SOR/2009-95, s. 1]
[Repealed, SOR/2009-95, s. 1]
[Repealed, SOR/96-278, s. 1]
[Repealed, 

[Repealed, SOR/2016-297, s. 12]
[Repealed, SOR/2016-297, s. 12]
Repealed, SOR/2004-254
Repealed, SOR/2004-254
[Repealed, SOR/2007-110, s. 1]
[Repealed, SOR/2007-110, s. 1]
[Repealed, 2014, c. 2, s. 72]
[Repealed, 2014, c. 2, s. 72]
[Repealed, SOR/2013-119, s. 267]
[Repealed, SOR/2013-119, s. 267]
[Repealed, SOR/2017-55, s. 1]
[Repealed, SOR/2017-55, s. 1]
[Repealed, SOR/2011-24, s. 1]
[Repealed, SOR/2011-24, s. 1]
[Repealed, SOR/2014-177, s. 4]
[Repealed, SOR/2014-177, s. 4]
[Repealed, SOR/2010-43, s. 1]
[Repealed, SOR/2010-43, s. 1]
[Repealed, SOR/2016-153, s. 1]
[Repealed, SOR/2016-153, s. 1]
[Repealed, SOR/2016-201, s. 3]
[Repealed, SOR/2016-201, s. 3]
[Repealed, SOR/2006-165, s. 3]
[Repealed, SOR/2006-165, s. 3]
[Repealed, SOR/2015-26, s. 89]
[Repealed, SOR/2015-26, s. 89]
[Repealed, SOR/2013-48, s. 12]
[Repealed, SOR/2013-48, s. 12]
[Repealed, SOR/2013-48, s. 13]
[Repealed, SOR/2013-48, s. 13]
[Repealed, SOR/2009-62, s. 1]
[Repealed, SOR/2009-62, s. 1]
[Repealed, SOR/2009-63, s. 1

[Repealed, SOR/2014-74, s. 1]
[Repealed, SOR/2009-322, s. 25]
[Repealed, SOR/2009-322, s. 25]
[Repealed, SOR/2009-322, s. 27]
[Repealed, SOR/2009-322, s. 27]
[Repealed, SOR/2016-93, s. 2]
[Repealed, SOR/2016-93, s. 2]
[Repealed, SOR/2015-19, s. 1]
[Repealed, SOR/2015-19, s. 1]
[Repealed, SOR/2018-11, s. 33]
[Repealed, SOR/2018-11, s. 33]
[Repealed, SOR/2010-83, s. 38]
[Repealed, SOR/2010-83, s. 38]
[Repealed, SOR/2013-20, s. 8]
[Repealed, SOR/2013-20, s. 8]
[Repealed, SOR/2009-157, s. 1]
[Repealed, SOR/2009-157, s. 1]
[Repealed, SOR/2011-261, s. 6]
[Repealed, SOR/2011-261, s. 6]
[Repealed, SOR/2010-89, s. 1]
[Repealed, SOR/2010-89, s. 1]
[Repealed, SOR/2016-38, s. 1]
[Repealed, SOR/2016-38, s. 1]
[Repealed, SOR/2014-104, s. 44]
[Repealed, SOR/2014-104, s. 44]
[Repealed, SOR/2009-31, s. 1]
[Repealed, SOR/2009-31, s. 1]
[Repealed, SOR/2012-239, s. 1]
[Repealed, SOR/2012-239, s. 1]
[Repealed, SOR/2007-163, s. 1]
[Repealed, SOR/2007-163, s. 1]
[Repealed, SOR/2012-285, s. 17]
[Repealed, SOR

[Repealed, SOR/2017-238, s. 1]
[Repealed, SOR/2017-238, s. 1]
[Repealed, SOR/2017-285, s. 2]
[Repealed, SOR/2017-285, s. 2]
[Repealed, SOR/62-113, s. 1]
[Repealed, SOR/62-113, s. 1]
[Repealed, SOR/69-30, s. 7]
[Repealed, SOR/69-30, s. 7]
[Repealed, SOR/2008-170, s. 1]
[Repealed, SOR/2008-170, s. 1]
[Repealed, SOR/2008-170, s. 2]
[Repealed, SOR/2008-170, s. 2]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2015-72, s. 1]
[Repealed, SOR/2015-72, s. 1]
[Repealed, SOR/2015-72, s. 1]
[Repealed, SOR/2015-72, s. 1]
[Repealed, SOR/93-167, s. 1]
[Repealed, SOR/93-167, s. 1]
[Revoked, SOR/85-299, s. 1]
[Revoked, SOR/85-299, s. 1]
[Repealed, SOR/2017-135, s. 1]
[Repealed, SOR/2017-135, s. 1]
[Repealed, SOR/2003-338, s. 2]
[Repealed, SOR/2003-338, s. 2]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2007-303, s. 44]
[Repeale

[Repealed, SOR/2017-58, s. 10]
[Repealed, SOR/2017-58, s. 10]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/88-68, s. 1]
[Repealed, SOR/88-68, s. 1]
[Repealed, SOR/2015-164, s. 15]
[Repealed, SOR/2015-164, s. 15]
[Repealed, SOR/91-522, s. 1(E)]
[Repealed, SOR/91-522, s. 1(E)]
[Repealed by operation of law, 1997, c. 20, s. 45
[Repealed by operation of law, 1997, c. 20, s. 45
[Repealed, SOR/2017-202, s. 2]
[Repealed, SOR/2017-202, s. 2]
[Repealed, SOR/2013-169, s. 2]
[Repealed, SOR/2013-169, s. 2]
[Revoked, SOR/92-582, s. 3]
[Revoked, SOR/92-582, s. 3]
[Repealed, SOR/2017-84, s. 1]
[Repealed, SOR/2017-84, s. 1]
[Revoked, SOR/93-240, s. 2]
[Revoked, SOR/93-240, s. 2]
[Repealed, SOR/2017-206, s. 1]
[Repealed, SOR/2017-206, s. 1]
[Repealed, SOR/2016-289, s. 12]
[Repealed, SOR/2016-289, s. 12]
[Repealed, SOR/2006-148, s. 1]
[Repealed, SOR/2006-148, s. 1]
[Revoked, SOR/92-152, s. 1]
[Revoked, SOR/92-152, s. 1]
[Repealed, SOR/2010-261, s. 44]
[Repealed, SOR/2010-26

[Repealed, SOR/2012-191, s. 11]
[Repealed, SOR/2012-191, s. 11]
[Repealed, SOR/2006-162, s. 4]
[Repealed, SOR/2006-162, s. 4]
[Repealed, SOR/2014-248, s. 10]
[Repealed, SOR/2014-248, s. 10]
[Repealed, SOR/2014-248, s. 13]
[Repealed, SOR/2014-248, s. 13]
[Repealed, SOR/2014-248, s. 16]
[Repealed, SOR/2014-248, s. 16]
[Repealed, SOR/2018-11, s. 2]
[Repealed, SOR/2018-11, s. 2]
[Repealed, SOR/93-253, s. 2]
[Repealed, SOR/93-253, s. 2]
[Repealed, SOR/2002-277, s. 18]
[Repealed, SOR/2002-277, s. 18]
[Repealed, SOR/2018-87, s. 1]
[Repealed, SOR/2018-87, s. 1]
[Repealed, SOR/2006-162, s. 6]
[Repealed, SOR/2006-162, s. 6]
 [Repealed, SOR/2002-186, s. 1]
 [Repealed, SOR/2002-186, s. 1]
[Repealed, SOR/96-551, ss. 3, 4(T)]
[Repealed, SOR/96-551, ss. 3, 4(T)]
[Repealed, SOR/2012-286, s. 41]
[Repealed, SOR/2012-286, s. 41]
[Repealed, SOR/2007-86, s. 204]
[Repealed, SOR/2007-86, s. 204]
[Repealed, SOR/2007-86, s. 205]
[Repealed, SOR/2007-86, s. 205]
[Repealed, SOR/2009-307, s. 1]
[Repealed, SOR/2009

[Repealed, SOR/2006-319, s. 5]
[Repealed, SOR/2006-319, s. 5]
[Repealed, SOR/2006-321, s. 5]
[Repealed, SOR/2006-321, s. 5]
[Repealed, SOR/2012-69, s. 136]
[Repealed, SOR/2012-69, s. 136]
[Repealed, SOR/97-48, s. 2]
[Repealed, SOR/97-48, s. 2]
[Repealed, SOR/97-43, s. 2]
[Repealed, SOR/97-43, s. 2]
[Repealed, SOR/97-41, s. 2]
[Repealed, SOR/97-41, s. 2]
[Repealed, SOR/97-42, s. 2]
[Repealed, SOR/97-42, s. 2]
[Repealed, SOR/97-40, s. 2]
[Repealed, SOR/97-40, s. 2]
[Repealed, SOR/97-47, s. 2]
[Repealed, SOR/97-47, s. 2]
[Repealed, SOR/2006-128, s. 1]
[Repealed, SOR/2006-128, s. 1]
[Repealed, SOR/97-44, s. 2]
[Repealed, SOR/97-44, s. 2]
[Repealed, SOR/2011-79, s. 54]
[Repealed, SOR/2011-79, s. 54]
[Repealed, SOR/97-45, s. 2]
[Repealed, SOR/97-45, s. 2]
[Repealed, SOR/97-46, s. 2]
[Repealed, SOR/97-46, s. 2]
[Repealed, SOR/2012-238, s. 1]
[Repealed, SOR/2012-238, s. 1]
[Repealed, SOR/2008-116, s. 12]
[Repealed, SOR/2008-116, s. 12]
[Repealed, SOR/2012-29, s. 1]
[Repealed, SOR/2012-29, s. 1

In [64]:
len(lst)

2062

In [67]:
print(n,count,n-count)

3209 1147 2062


In [54]:
print(d.keys())


dict_keys(['shorttitle', 'longtitle', 'registrationyear', 'consolidationyear', 'regnalyears', 'xrefxternal', 'xrefinternal', 'content'])


In [70]:
import pandas as pd
regulations = pd.DataFrame(lst)

In [77]:
regulations.content.values

array(['C.R.C., c. 10 AERONAUTICS ACT Regulations Prescribing Compensation for Bodily Injury or Death Resulting from Flights Undertaken in the Course of Duty. Short Title. These Regulations may be cited as the  Flying Accidents Compensation Regulations . Interpretation. In these Regulations, aircraft  means any machine used or designed for navigation of the air but does not include a machine designed to derive support in the atmosphere from reaction against the earth’s surface of air expelled from the machine; ( aéronef ) civil aviation inspector  means an employee who is required to conduct flight tests to determine the competency of flight crew personnel and to inspect commercial air operations, or an employee who is required to monitor in-flight cabin procedures in use in commercial air operations; ( inspecteur de l’aviation civile ) Commission  means the Canadian Pension Commission established by section 3 of the  Pension Act ; ( Commission ) employee  means any person who is emplo

In [79]:
regulations.to_csv("regulation.csv")